In [ ]:
import astropy
from astropy.coordinates import SkyCoord
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from PIL import Image
from io import BytesIO

from astropy.table import Table, join
from astropy.io import ascii

# set width for pprint
astropy.conf.max_width = 150

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

### Useful functions

* The `hcvcone(ra,dec,radius [,keywords])` function searches the HCV catalog near a position.
* The `hcvsearch()` function performs general non-positional queries.
* The `hcvmetadata()` function gives information about the columns available in a table.

In [ ]:
hscapiurl = "https://catalogs.mast.stsci.edu/api/v0.1/hsc"


def hcvcone(ra, dec, radius, table="hcvsummary", release="v3", format="csv", magtype="magaper2",
            columns=None, baseurl=hscapiurl, verbose=False, **kw):
    """Do a cone search of the HSC catalog (including the HCV)
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return hcvsearch(table=table, release=release, format=format, magtype=magtype,
                     columns=columns, baseurl=baseurl, verbose=verbose, **data)


def hcvsearch(table="hcvsummary", release="v3", magtype="magaper2", format="csv",
              columns=None, baseurl=hscapiurl, verbose=False, **kw):
    """Do a general search of the HSC catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'numimages.gte':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    if format not in ("csv", "votable", "json"):
        raise ValueError("Bad value for format")
    url = f"{cat2url(table, release, magtype, baseurl=baseurl)}.{format}"
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in hcvmetadata(table, release, magtype)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError(f"Some columns not found in table: {', '.join(badcols)}")
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = f"[{','.join(columns)}]"

    # either get or post works
    # r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def hcvmetadata(table="hcvsummary", release="v3", magtype="magaper2", baseurl=hscapiurl):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    url = f"{cat2url(table,release,magtype,baseurl=baseurl)}/metadata"
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'], x['type'], x['description']) for x in v],
                names=('name', 'type', 'description'))
    return tab


def cat2url(table="hcvsummary", release="v3", magtype="magaper2", baseurl=hscapiurl):
    """Return URL for the specified catalog and table
    
    Parameters
    ----------
    table (string): hcvsummary, hcv, summary, detailed, propermotions, or sourcepositions
    release (string): v3 or v2
    magtype (string): magaper2 or magauto (only applies to summary table)
    baseurl: base URL for the request
    
    Returns a string with the base URL for this request
    """
    checklegal(table, release, magtype)
    if table == "summary":
        url = f"{baseurl}/{release}/{table}/{magtype}"
    else:
        url = f"{baseurl}/{release}/{table}"
    return url


def checklegal(table, release, magtype):
    """Checks if this combination of table, release and magtype is acceptable
    
    Raises a ValueError exception if there is problem
    """
    
    releaselist = ("v2", "v3")
    if release not in releaselist:
        raise ValueError(f"Bad value for release (must be one of {', '.join(releaselist)})")
    if release == "v2":
        tablelist = ("summary", "detailed")
    else:
        tablelist = ("summary", "detailed", "propermotions", "sourcepositions", "hcvsummary", "hcv")
    if table not in tablelist:
        raise ValueError(f"Bad value for table (for {release} must be one of {', '.join(tablelist)})")
    if table == "summary":
        magtypelist = ("magaper2", "magauto")
        if magtype not in magtypelist:
            raise ValueError(f"Bad value for magtype (must be one of {', '.join(magtypelist)})")

## Variable objects near Adell 2390

### Use SkyCoord name resolver to get the position of Adell 2390

In [ ]:
target = 'Abell 2390'
coord_adell2390 = SkyCoord.from_name(target)

ra_adell2390 = coord_adell2390.ra.degree
dec_adell2390 = coord_adell2390.dec.degree
print(f'ra: {ra_adell2390}\ndec: {dec_adell2390}')

In [ ]:
radius = 0.2 # degrees
cone = hcvcone(ra_adell2390, dec_adell2390, radius, table="summary")
tab = ascii.read(cone)
df = tab.to_pandas()
df.head()

In [ ]:
g = sns.relplot(data=df, x='MatchRA', y='MatchDec', color='purple', height=10, hue='TargetName', alpha=.5)

for ax in g.axes.ravel():
    ax.invert_xaxis()

## Helpful Information

- https://stackoverflow.com/q/54116787/7758804
- https://stackoverflow.com/q/39008262/7758804
- https://openastronomy.org/rcsc18/chapters/13-images-in-astronomy/01-images-in-astronomy
- https://learn.astropy.org/
- https://outerspace.stsci.edu/display/PANSTARRS/PS1+Image+Cutout+Service
  - [Image][1]
  - http://ps1images.stsci.edu/cgi-bin/ps1cutouts
- [Get Image Notebook][2]
  
  
  
  [1]: https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?red=/rings.v3.skycell/1756/040/rings.v3.skycell.1756.040.stk.i.unconv.fits&blue=/rings.v3.skycell/1756/040/rings.v3.skycell.1756.040.stk.g.unconv.fits&green=/rings.v3.skycell/1756/040/rings.v3.skycell.1756.040.stk.r.unconv.fits&x=328.397920&y=17.686670&size=2880&wcs=1&asinh=True&autoscale=99.750000&output_size=1024
  [2]: https://ps1images.stsci.edu/ps1image.html

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.utils.data import get_pkg_data_filename

In [ ]:
SkyCoord.from_name('Abell 2390')

# casjobs Implementation

In [ ]:
import astropy
from astropy.coordinates import SkyCoord
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO

from astropy.table import Table, join

# check that version of mastcasjobs is new enough
# we are using some features not in version 0.0.1
from pkg_resources import get_distribution
from packaging.version import Version as V

assert V(get_distribution("mastcasjobs").version) >= V('0.0.2'), """
A newer version of mastcasjobs is required.
Update mastcasjobs to current version using this command:
pip install --upgrade git+git://github.com/rlwastro/mastcasjobs@master
"""

import mastcasjobs

# set width for pprint
astropy.conf.max_width = 150

In [ ]:
HSCContext = "HSCv3"

Set up Casjobs environment.

In [ ]:
import getpass
if not os.environ.get('CASJOBS_USERID'):
    os.environ['CASJOBS_USERID'] = input('Enter Casjobs UserID:')
if not os.environ.get('CASJOBS_PW'):
    os.environ['CASJOBS_PW'] = getpass.getpass('Enter Casjobs password:')

## Variable objects near Adell 2390

### Use SkyCoord name resolver to get the position of Adell 2390

In [ ]:
target = 'Abell 2390'
coord_adell2390 = SkyCoord.from_name(target)

ra_adell2390 = coord_adell2390.ra.degree
dec_adell2390 = coord_adell2390.dec.degree
print(f'ra: {ra_adell2390}\ndec: {dec_adell2390}')

# TODO: Not able to get this to download any data

In [ ]:
HSCContext = "HSCv3"

DBtable = "summary"
jobs = mastcasjobs.MastCasJobs(context="MyDB")

# drop table if it already exists
jobs.drop_table_if_exists(DBtable)

# get main information
radius = 1800.0 # arcsec
query = f"""
select m.MatchID, m.GroupID, m.SubGroupID, m.RA, m.Dec,
   m.AutoClass, m.ExpertClass, m.NumFilters,
   f.Filter, f.FilterDetFlag, f.VarQualFlag, f.NumLC, 
   f.MeanMag, f.MeanCorrMag, f.MAD, f.Chi2
into mydb.{DBtable}
from SearchHCVMatchID({ra_adell2390},{dec_adell2390},{radius}) s
join HCVmatch m on m.MatchID=s.MatchID
join HCVfilter f on f.MatchID=s.MatchID and (f.Filter='ACS_F475W' or f.Filter='ACS_F814W')
"""

t0 = time.time()
results = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print(f"Completed in {(time.time()-t0):.1f} sec")
print(results)

# fast retrieval using special MAST Casjobs service
tab = jobs.fast_table(DBtable, verbose=True)

# clean up the output format
tab['MeanMag'].format = "{:.3f}"
tab['MeanCorrMag'].format = "{:.3f}"
tab['MAD'].format = "{:.4f}"
tab['Chi2'].format = "{:.4f}"
tab['RA'].format = "{:.6f}"
tab['Dec'].format = "{:.6f}"

# show some of the variable sources
tab[tab['AutoClass'] > 0]

# A Hubble Source Catalog (HSC) Use Case

- [Example 6][1]: Using the Discovery Portal to study the Red Sequence in a Galaxy Cluster
- (The Red Sequence in the Galaxy Cluster Abell 2390)


  [1]: https://archive.stsci.edu/hst/hsc/help/use_case_6_v1.html

GOAL: This tutorial shows you how to use the [MAST Discovery Portal][1] to create a Color-Magnitude diagram of extended sources.

SCIENCE CASE: The science case is to isolate the red sequence - a color-magnitude relation for elliptical and lenticular galaxies in clusters of galaxies - in a cluster well observed by HST (i.e. Abell 2390 at z=0.2; see [Gladders and Yee 2000, AJ, 120, 2148][2]). Aperture corrections and extinction corrections are also performed.
Other potential use cases could include testing cluster evolution via the change in the slope of the red sequence, and identifying other clusters with pronounced red sequences.


  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: https://ui.adsabs.harvard.edu/abs/2000AJ....120.2148G/abstract

## Step 1 - Go to the [MAST Discovery Portal][1].

Use the pull down menu under Select Collection to choose the HSC (<span style="color:blue;">blue</span>). Enter the name of the cluster (or if you prefer the coordinates) and search radius (i.e. Abell 2390 r=0.2d) in the Search box (<span style="color:green;">green</span>). Perform the search by just hitting a carriage return. The results are displayed in the List of Objects, while the AstroView window shows the objects against the DSS image. The left column is a series of Filters that can be used to refine the data selected.

![redesequence_1][2]


  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: screenshots/redsequence_1.png

In [ ]:
target = 'Abell 2390'
coord_adell2390 = SkyCoord.from_name(target)

ra_adell2390 = coord_adell2390.ra.degree
dec_adell2390 = coord_adell2390.dec.degree
print(f'ra: {ra_adell2390}\ndec: {dec_adell2390}')

In [ ]:
radius = 0.2 # degrees
cone = hcvcone(ra_adell2390, dec_adell2390, radius, table="summary")
tab = ascii.read(cone)
df = tab.to_pandas()
df.head()

In [ ]:
g = sns.relplot(data=df, x='MatchRA', y='MatchDec', color='purple', height=10, hue='TargetName', alpha=.5)

for ax in g.axes.ravel():
    ax.invert_xaxis()

## Step 2 - Refine the sample.

The HSC includes both compact (point) and extended sources, as well as a few residual cosmic rays and image artifacts. Since we are looking for galaxies (i.e. extended sources), we want to remove as many point sources as possible. Scroll down the Filters section (<span style="color:blue;">blue</span>) to the CI (Concentration Index) area and set the lower limit to be 1.3 by either moving the slider or typing in the value (<span style="color:green;">green</span>). Note that about 1/3 of the objects have been rejected (<span style="color:orange;">orange</span>).

![redsequence_2][1]


  [1]: screenshots/redsequence_2.png

## Step 3 - Determine the HSC magnitudes corrections.

To make our Color-Magnitude diagram, we need to correct the magnitude values for the V (W2_F555W = WFPC2 F555W) and I (W2_F814W = WFPC2 F814W) to infinite aperture magnitudes for better comparison, and correct for Galactic extinction along the line of sight.

Approximate aperture corrections can be taken from the [aperture corrections table][1], or estimated from the encircled energy curves provided by the instrument teams. For both filters, the aperture corrections are 0.17 mag, or:

W2_F555W_Inf = W2_F555W - 0.17
W2_F814W_Inf = W2_F814W - 0.17.

Extinction correction is best estimated using an absorption calculator, like [Doug's Excellent Absorption Law Calculator][2]. The HSC table includes a column labeled Extinction, which is the Schlegel, Finkbeiner, & Davis 1998 ([ApJ, 500, 525, 1998][3]) E(B-V)=0.114 map value at that position. Using the absorption law calculator, with R_V = 3.1, and A_V = E(B-V) x R_V = 0.353, and assuming target wavelength equal to the pivot wavelength for each filter, or:

W_V = 0.535 (um)
W_I = 0.820 (um).

We get the following extinction corrections for each filter:

W2_F555W_Ext = W2_F555W - 0.36
W2_F814W_Ext = W2_F814W - 0.20,

and arrive at the final corrections for each filter:

W2_F555W_Cor = W2_F555W - 0.53
W2_F814W_Cor = W2_F814W - 0.37.


  [1]: https://archive.stsci.edu/hst/hsc/help/FAQ/aperture_corrections.txt
  [2]: http://dogwood.physics.mcmaster.ca/Acurve.html
  [3]: http://adsabs.harvard.edu/abs/1998ApJ...500..525S